# The 48-dim *Cauchy* example in the [GBS paper](http://proceedings.mlr.press/v118/jia20a.html)

*last tested with bayesfast commit 5810931*

In [1]:
import bayesfast as bf
import numpy as np
from threadpoolctl import threadpool_limits
threadpool_limits(1) # TODO: implement a bayesfast global thread controller

In [2]:
D = 48 # number of dims
a = 5.
lower = np.full(D, -100.) # lower bound of the prior
upper = np.full(D, 100.) # upper bound of the prior
bound = np.array((lower, upper)).T
diff = bound[:, 1] - bound[:, 0]
const = np.sum(np.log(diff)) # normalization of the flat prior

def logp(x):
    _a = 1 / ((x + a)**2 + 1)
    _b = 1 / ((x - a)**2 + 1)
    return (np.sum(np.log(_a + _b), axis=-1) + x.shape[-1] * np.log(0.5 / np.pi)
            - const)

def grad(x):
    _a = 1 / ((x + a)**2 + 1)
    _b = 1 / ((x - a)**2 + 1)
    return -2 * ((x + a) / ((x + a)**2 + 1)**2 + 
                 (x - a) / ((x - a)**2 + 1)**2) / (_a + _b)

In [3]:
bf.utils.random.set_generator(48) # set up the global random number generator
bf.utils.parallel.set_backend(8) # set up the global parallel backend
den = bf.DensityLite(logp=logp, grad=grad, input_size=D, input_scales=bound,
                     hard_bounds=True)
sample_trace = {'n_chain': 8, 'n_iter': 2500, 'n_warmup': 1000}
rec = bf.Recipe(density=den, sample={'sample_trace': sample_trace},
                post={'evidence_method': 'GBS'})

In [4]:
rec.run()


 *** StaticSample: returning the #0 SampleStep. *** 



/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:183: RuntimeWarning:  CHAIN #6 : divide by zero encountered in log
  return np.sum(np.log(np.abs(self.to_original_grad(x_trans))),
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:930: RuntimeWarning:  CHAIN #6 : divide by zero encountered in true_divide
  _grad += self.to_original_grad2(x) / _tog


 CHAIN #1 : sampling proceeding [ 500 / 2500 ], last 500 samples used 29.32 seconds. (warmup)
 CHAIN #4 : sampling proceeding [ 500 / 2500 ], last 500 samples used 30.69 seconds. (warmup)
 CHAIN #0 : sampling proceeding [ 500 / 2500 ], last 500 samples used 32.25 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 500 / 2500 ], last 500 samples used 37.95 seconds. (warmup)
 CHAIN #2 : sampling proceeding [ 500 / 2500 ], last 500 samples used 39.82 seconds. (warmup)
 CHAIN #5 : sampling proceeding [ 500 / 2500 ], last 500 samples used 40.58 seconds. (warmup)
 CHAIN #6 : sampling proceeding [ 500 / 2500 ], last 500 samples used 43.84 seconds. (warmup)
 CHAIN #7 : sampling proceeding [ 500 / 2500 ], last 500 samples used 44.10 seconds. (warmup)
 CHAIN #4 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 23.53 seconds. (warmup)
 CHAIN #1 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 27.22 seconds. (warmup)
 CHAIN #0 : sampling proceeding [ 1000 / 2500 ], last 500 

/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/glo


 ***** PostStep finished. ***** 



In [5]:
rec.get()._fields

('samples',
 'weights',
 'weights_trunc',
 'logp',
 'logq',
 'logz',
 'logz_err',
 'x_p',
 'x_q',
 'logp_p',
 'logq_q',
 'trace_p',
 'trace_q',
 'n_call',
 'x_max',
 'f_max')

In [6]:
rec.get().logz, rec.get().logz_err # fiducial value: logz = -254.627

(-254.6878565502715, 0.09192743651703258)